<a href="https://colab.research.google.com/github/babaroscopy/Thesis/blob/main/Thesis_RP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
import urllib.request

# Define the URL to download the checkpoint file from
url = 'https://dl.fbaipublicfiles.com/fasterrcnn/faster_rcnn_r50_fpn_coco/checkpoints/checkpoint.pth'

# Define the path to save the checkpoint file to
checkpoint_path = '/path/to/model/checkpoint'

# Download the checkpoint file and save it to the specified path
urllib.request.urlretrieve(url, checkpoint_path)

HTTPError: ignored

In [ ]:

def get_model(num_classes):
    # Load a pre-trained model for classification and return
    #model = torch.hub.load('pytorch/vision:v0.9.0', 'fasterrcnn_resnet50_fpn', pretrained=True)
    # load model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

In [ ]:
'''
# load model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# set to evaluation mode
model.eval()
'''



/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
def process_image(image_path, threshold=0.5, n_proposals=2000, max_regions=4):
    # Load the image
    image = Image.open(image_path).convert("RGB")
    # Define the transforms to be applied on the image
    transform = T.Compose([T.ToTensor()])

    # Apply the transforms to the image
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)

    # Load the model
    model = get_model(num_classes=2)
    model.eval()

    # Generate the region proposals using selective search
    regions = selective_search(image_tensor, n_proposals, max_regions, min_size)

    # Draw the regions on the original image
    image_draw = cv2.imread(image_path)
    for region in regions:
        x, y, w, h = region
        cv2.rectangle(image_draw, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Detect the object in the regions
    predictions = detect_objects(image_tensor, regions, model, threshold)

    # Get the selected regions based on confidence score
    selected_regions = []
    for prediction in predictions:
        if prediction['score'] >= threshold:
            selected_regions.append(prediction['box'])

    return selected_regions, image_draw


In [ ]:
def selective_search_regions(image, n_proposals=2000, max_proposals=10000, min_size=2):
    # Extract region proposals
    img_lbl, regions = selectivesearch.selective_search(
        image, scale=n_proposals, sigma=0.9, min_size=min_size)
    regions = [r for r in regions if r['size'] > min_size]

    # Extract color histograms of regions
    hist = compute_color_histograms(image, regions)
    hist = pd.DataFrame(hist)

    # Extract texture features of regions
    texture = compute_texture_features(image, regions)
    texture = pd.DataFrame(texture)

    # Concatenate color histograms and texture features into feature vectors
    features = pd.concat([hist, texture], axis=1)

    # Normalize feature vectors
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Cluster feature vectors using KMeans
    n_clusters = int(np.sqrt(len(regions) / (max_proposals / n_proposals)))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(scaled_features)
    labels = kmeans.labels_

    # Select regions with highest cluster scores
    selected_regions = []
    for i in range(n_clusters):
        indices = np.where(labels == i)[0]
        cluster_regions = [regions[j] for j in indices if j < len(regions)]
        cluster_scores = [np.max(labels == i)] * len(cluster_regions)
        selected_regions.extend(zip(cluster_regions, cluster_scores))

    # Sort selected regions by cluster score
    selected_regions = sorted(selected_regions, key=lambda x: x[1], reverse=True)

    # Return selected regions
    return selected_regions


In [ ]:
def extract_features(image, regions):
    feature_extractor = cv2.xfeatures2d.SIFT_create()
    region_features = []
    for (x, y, w, h) in regions:
        region_image = image[y:y+h, x:x+w]
        gray_region = cv2.cvtColor(region_image, cv2.COLOR_BGR2GRAY)
        gray_region = cv2.convertScaleAbs(gray_region)
        _, descriptors = feature_extractor.detectAndCompute(gray_region, None)
        if descriptors is not None:
            region_features.append(descriptors)
    return region_features


In [ ]:
'''
def detect_objects(image, regions, model, threshold):
    # Define the list to store the predictions
    predictions = []

    # Convert the regions to tensors
    boxes = []
    for region in regions:
        x, y, w, h = region
        box = [x, y, x+w, y+h]
        boxes.append(box)
    boxes = torch.tensor(boxes)

    # Detect the objects in the regions using the model
    with torch.no_grad():
        outputs = model(image, boxes)
        predictions = outputs[0]['boxes'].cpu().numpy()
        scores = outputs[0]['scores'].cpu().numpy()
        labels = outputs[0]['labels'].cpu().numpy()

    # Store the predictions in a list
    predictions = []
    for i in range(len(predictions)):
        prediction = {'box': predictions[i], 'score': scores[i], 'label': labels[i]}
        predictions.append(prediction)

    return predictions
'''

In [ ]:
def draw_regions(image_path, regions):
    # Load the image
    image = cv2.imread(image_path)

    # Draw rectangles around the selected regions
    for region in regions:
        x, y, w, h = region
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the image with the selected regions
    cv2.imshow('Selected Regions', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

import cv2
import numpy as np
import imutils
from skimage import io

def detect_objects(image_path, output_path, threshold=0.5, n_proposals=2000, max_regions=4):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Extract region proposals
    regions, _ = selective_search(image_rgb, scale=500, sigma=0.9)
    regions = [r for r in regions if r['size'] > 100]
    
    # Extract features for the region proposals
    features = extract_features(image_rgb, regions)
    
    # Perform clustering on the features
    n_clusters = int(np.sqrt(len(regions)))
    if n_clusters == 0:
        return []
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(features)
    labels = kmeans.labels_
    
    # Select the top regions based on their scores
    scores = kmeans.transform(features).min(axis=1)
    regions_scores = [(regions[i], scores[i]) for i in range(len(regions))]
    regions_scores = sorted(regions_scores, key=lambda x: x[1], reverse=True)
    selected_regions = []
    for i in range(n_clusters):
        indices = np.where(labels == i)[0]
        cluster_regions = [regions[j] for j in indices if j < len(regions)]
        cluster_scores = [np.max(labels == i)] * len(cluster_regions)
        selected_regions.extend(zip(cluster_regions, cluster_scores))

    selected_regions = sorted(selected_regions, key=lambda x: x[1], reverse=True)
    selected_regions = [r for r, _ in selected_regions][:max_regions]
    
    # Draw the selected regions on the original image and save it
    img = cv2.imread(image_path)
    for region, score in selected_regions:
        x, y, w, h = region['rect']
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imwrite(output_path, img)
    
    return selected_regions


In [ ]:
image_path = '/content/Dataset/10.jpg'
output_path = '/content/output.jpg'
selected_regions = detect_objects(image_path, output_path, threshold=0.5, n_proposals=2000, max_regions=2)


AttributeError: ignored